# Analyzing the loaded data

Now that the data is loaded, let's have some fun in analyzing the data.

## Loading the packages

We need to load the libraries one more time

In [13]:
# Install the rare packages needed
!pip install neo4j-driver
!pip install graphdatascience==1.0.0

In [14]:
# Importing the needed packages
import pandas as pd
import numpy as np
from graphdatascience import GraphDataScience
from myClassConnectToNeo4J import Neo4JConnection

In [15]:
# Connecting to the database (using localhost because I'm too cheap right now to get a Sandbox :P )
gds = Neo4JConnection()

In [16]:
# Print out the stats to confimr the data is loaded
df = gds.printStatistics()
df.T

,0
labelCount,6
relTypeCount,7
propertyKeyCount,26
nodeCount,443
relCount,1069
labels,"{'Type': 4, 'Skill': 278, 'Stages': 58, 'Chapt..."
relTypes,"{'(:Stages)-[:located_at]->()': 58, '()-[:loca..."
relTypesCount,"{'in_chapter': 58, 'has_skill': 279, 'prerequi..."
stats,"{'relTypeCount': 7, 'propertyKeyCount': 26, 'l..."


### Running some code

In [17]:
# Print all the nodes and the count
gds.runCypher(
    """
MATCH (n)
RETURN labels(n)[0] AS label, count(*) AS count;
"""
)

,label,count
0,Skill,278
1,Job,45
2,Type,4
3,Chapter,4
4,Locations,54
5,Stages,58


In [18]:
# Print all the relationships and the count
gds.runCypher(
    """
MATCH ()-[r]->()
RETURN type(r) AS label, count(r) as count;
    """
)

,label,count
0,is_type,279
1,has_skill,279
2,prerequisite_of,23
3,has_enemy,315
4,in_chapter,58
5,next_battle,57
6,located_at,58


#### Question 1

How many Skills are identical in other jobs?

Looks like Fly is both in Bard and Dancer.

![Graph image](images/graph_1.png)

In [19]:
gds.runCypher(
    """
MATCH (j1:Job)-[r1:has_skill]->(s:Skill)<-[r2:has_skill]-(j2:Job)
RETURN j1.title, s.title as SkillTitle, j2.title;
    """
)

,j1.title,SkillTitle,j2.title
0,Dancer,Fly,Bard
1,Bard,Fly,Dancer


#### Question 2

What's the breakdown of types and the count of the number of skills?

Lots of Commands which make sense as those are the actions.

In [20]:
gds.runCypher(
    """
MATCH (n:Type)-[r1]-(s:Skill)
RETURN n.title AS label, count(s) AS count
"""
)

,label,count
0,Command,200
1,Reaction,30
2,Support,28
3,Movement,21


### Question 3

What skills are connected to the class Monk, and what is the job points needed? And what's the total number of JP to get all the jobs?

Wow, that's a lot of rock tossing to get those tasks.

![Graph image](images/graph_3.png)

In [21]:
gds.runCypher(
    """
match (j:Job{title:"Monk"}) -[t:has_skill]->(s:Skill)
RETURN j.title AS Job,s.title AS skill,toInteger(t.jp) as jp_needed
ORDER BY toInteger(t.jp) desc
    """
)

,Job,skill,jp_needed
0,Monk,Hamedo,1200
1,Monk,Earth Slash,600
2,Monk,HP Restore,500
3,Monk,Revive,500
4,Monk,Chakra,350
5,Monk,Wave Fist,300
6,Monk,Secret Fist,300
7,Monk,Counter,300
8,Monk,Repeating Fist,300
9,Monk,Move-HP Up,300


In [22]:
gds.runCypher(
    """
match (j:Job{title:"Monk"}) -[t:has_skill]->(s:Skill)
RETURN j.title, sum(toInteger(t.jp)) as TotalJP
    """
)

,j.title,TotalJP
0,Monk,5200


### Question 4

What bad guys are going to be on Sweegy Woods?

Oh man, not the bombs. I hate those things.

![Graph image](images/graph_4.png)

In [23]:
gds.runCypher(
    """
match (s:Stages{chronological: "4"})-[t:has_enemy]->(j:Job)
RETURN s.chronological, s.title, s.win_condition, j.title, t.level
    """
)

,s.chronological,s.title,s.win_condition,j.title,t.level
0,4,Sweegy Woods,Defeat all enemies,Red Panther,3
1,4,Sweegy Woods,Defeat all enemies,Black Goblin,3
2,4,Sweegy Woods,Defeat all enemies,Goblin,3
3,4,Sweegy Woods,Defeat all enemies,Goblin,3
4,4,Sweegy Woods,Defeat all enemies,Bomb,3
5,4,Sweegy Woods,Defeat all enemies,Bomb,3


#### Question 5

Which levels have Time Mage enemies?

A lot, apparently, and it's time to cry. They keep slowing and stopping me. Ugh.

![Graph image](images/graph_5.png)

In [24]:
gds.runCypher(
    """
match (s:Stages)-[t:has_enemy]->(j:Job{title:"Time Mage"})
RETURN s.chronological as Level, s.title, s.win_condition, j.title, t.level as EnemyLevel
ORDER BY toInteger(s.chronological)
    """
)

,Level,s.title,s.win_condition,j.title,EnemyLevel
0,8,Lenalia Plateau,Defeat target,Time Mage,5
1,19,Golgorand Execution Site,Defeat all enemies,Time Mage,14
2,19,Golgorand Execution Site,Defeat all enemies,Time Mage,15
3,24,Underground Book Storage Second Floor,Defeat all enemies,Time Mage,22
4,24,Underground Book Storage Second Floor,Defeat all enemies,Time Mage,23
5,29,Yuguo Woods,Defeat all enemies,Time Mage,27
6,29,Yuguo Woods,Defeat all enemies,Time Mage,28
7,53,Underground Book Storage Fifth Floor,Defeat target,Time Mage,52
8,54,Murond Death City,Defeat target,Time Mage,52
9,54,Murond Death City,Defeat target,Time Mage,51
